# Information Retrieval Baseline Code

## 대회 설명
Information Retrieval 경진대회에 오신 여러분 환영합니다! 🎉 <br>
본 대회에서는 과학 상식에 대한 멀터턴 질문에 대하여 RAG 기술을 활용하여 질문에 적합한 레퍼런스를 추출하고 이를 토대로 답변을 생성하는 어플리케이션을 구현해 봅니다. <br>
LLM은 GPT-3.5 버전을 이용하고, 레퍼런스 추출을 위한 검색엔진 구축은 Elasticsearch를 활용합니다.

## 데이터셋 준비
* 질문에 대한 레퍼런스로 활용될 수 있는 4272개의 과학 상식을 다루는 문서
 * Open Ko LLM Leaderboard에 들어가는 Ko-H4 데이터 중 MMLU, ARC 데이터를 기반으로 생성
 * Question과 answer를 paraphrase (question과 answer를 토대로 GPT-4 활용하여 document 생성)
 * 예시)
  ```
  수소, 산소, 질소 가스의 혼합물에서 평균 속도가 가장 빠른 분자는 수소입니다
. 수소 분자는 가장 가볍고 작은 원자로 구성되어 있기 때문에 다른 분자들보다 더 빠르게 움직>입니다. 이러한 이유로 수소 분자는 주어진 온도에서 가장 빠른 평균 속도를 가지고 있습니다. >수소 분자는 화학 반응에서도 활발하게 참여하며, 수소 연료로도 널리 사용됩니다. 따라서 수소 분자는 주어진 온도에서 평균 속도가 가장 빠른 분자입니다.
  ```

* 가상의 질문 220개
 * 과학 상식 질문 또는 일상 대화 메시지를 수동으로 생성
 * 220개 중 20개는 멀티턴 대화 형태의 질문, 20개는 일상 칫챗 대화
  * 예시)
  ```
{"eval_id": 0, "msg": [{"role": "user", "content": "나무의 분류에 대해 조사해 보기 위한 방
법은?"}], "refs": ["c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d"]}
{"eval_id": 1, "msg": [{"role": "user", "content": "각 나라에서의 공교육 지출 현황에 대해 알려줘."}], "refs": ["79c93deb-fe60-4c81-8d51-cb7400a0a156"]}
{"eval_id": 2, "msg": [{"role": "user", "content": "기억 상실증 걸리면 너무 무섭겠다."}, {"role": "assistant", "content": "네 맞습니다."}, {"role": "user", "content": "어떤 원인 때
문에 발생하는지 궁금해."}], "refs": ["25de4ffd-cee4-4f27-907e-fd6b802c6ede"]}
  ```

## 검색엔진 준비 - Elasticsearch




In [10]:
# 엘라스틱서치의 데몬 인스턴스 만들기
import os
import json
from elasticsearch import Elasticsearch, helpers
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(60)

In [11]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon    349415  345106  4 14:27 ?        00:00:02 /data/ephemeral/scientific-knowledge-question-answering/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/data/ephemeral/scientific-knowledge-question-answering/elasticsearch-8.8.0 -Des.path.conf=/data/ephemeral/scientific-knowledge-question-answering/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /data/ephemeral/scientific-knowledge-question-answering/elasticsearch-8.8.0/lib/*:/data/ephemeral/scientific-knowledge-question-answering/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon    349473  349415 84 14:27 ?        00:00:49 /data/ephemeral/scientific-knowledge-question-answering/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.he

In [12]:
import os
import json
from elasticsearch import Elasticsearch, helpers

es_username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
es_password = 'rJxWGNBEM=2STcJ*_lBZ'

# Elasticsearch client 생성
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="./elasticsearch-8.8.0/config/certs/http_ca.crt")

# Elasticsearch client 정보 확인
print(es.info())

{'name': 'instance-11760', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'YI6CVvSVQfe3LuGlh_CC8Q', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [13]:
from sentence_transformers import SentenceTransformer

# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")


# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [14]:
# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        "content": {"type": "text", "analyzer": "nori"},
        "embeddings": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}


In [15]:
# settings, mappings 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", settings, mappings)

In [16]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("./data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

# 생성한 임베딩을 색인할 필드로 추가
for doc, embedding in zip(docs, embeddings):
    doc["embeddings"] = embedding.tolist()
    index_docs.append(doc)

# 'test' 인덱스에 대량 문서 추가
ret = bulk_add("test", index_docs)

# 색인이 잘 되었는지 확인 (색인된 총 문서수가 출력되어야 함)
print(ret)


batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200
(4272, [])


In [17]:
# 검색엔진에 색인이 잘 되었는지 테스트하기 위한 질의
test_query = "금성이 다른 행성들보다 밝게 보이는 이유는 무엇인가요?"

In [18]:
# 역색인을 사용하는 검색 예제
search_result_retrieve = sparse_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 33.62181 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 18.934727 source: 금성은 태양계의 두 번째로 가까운 행성입니다. 이 행성의 대략적인 나이는 7억 5천만 년으로 추정됩니다. 금성은 지구와 매우 비슷한 크기와 구성을 가지고 있으며, 약 90% 이상이 이산화탄소로 이루어져 있습니다. 이 행성은 매우 뜨거운 온도와 압력을 가지고 있어서 인간이 살 수 있는 환경이 아닙니다. 금성의 대기는 두꺼워서 태양의 열을 가두고 있어서 행성의 표면은 평균 온도가 약 450도로 매우 뜨거운 상태입니다. 또한, 금성은 자전 속도가 매우 빠르기 때문에 하루가 지구의 약 243일과 같습니다. 이러한 특징들로 인해 금성은 우리 태양계에서 가장 가혹한 환경을 가진 행성 중 하나로 알려져 있습니다.
score: 18.297876 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.


In [19]:
# Vector 유사도 사용한 검색 예제
search_result_retrieve = dense_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])

score: 0.0047387844 source: 금성이 다른 행성들보다 더 밝게 보이는 이유는 지구 쪽으로 가장 많은 햇빛을 반사하기 때문입니다. 케빈은 맑은 밤에 하늘을 관찰하고 있습니다. 그는 맨눈으로 금성, 화성, 목성, 토성을 볼 수 있습니다. 금성은 햇빛을 많이 반사하기 때문에 다른 행성들보다 더 밝게 보입니다. 이는 금성의 표면이 반사율이 높기 때문입니다. 금성은 태양으로부터 받은 햇빛을 표면에 반사하여 지구에서 관찰하기 쉽게 만듭니다. 따라서 케빈은 맑은 밤에 금성을 더 밝게 볼 수 있습니다.
score: 0.004689847 source: 금성은 태양계에서 가장 가까운 행성 중 하나입니다. 그러나 화성이나 지구처럼 계절이 없는 이유는 금성의 자전축이 태양계의 평면에 거의 수직이기 때문입니다. 자전축이 수직이기 때문에 금성은 태양으로부터 받는 햇빛의 양이 일정하게 유지됩니다. 이로 인해 금성은 계절 변화가 없으며 항상 일정한 온도를 유지합니다. 이러한 환경은 생명체에게는 적합하지 않을 수 있지만, 금성의 특이한 기후 조건은 우주 탐사에 대한 연구에 많은 도움을 주고 있습니다. 금성은 여전히 우리에게 알려지지 않은 많은 비밀을 품고 있으며, 미래에 더 많은 연구와 탐사가 이루어질 것으로 기대됩니다.
score: 0.004264912 source: 소행성대가 위치한 곳에는 행성이 없는 이유는 목성과의 공명으로 인해 물질이 모이는 것을 방해하여 행성을 형성하지 못했기 때문입니다. 소행성대는 태양계 내부 행성들과 외부 행성들 사이에 위치한 영역으로, 많은 소행성들이 모여있는 지역입니다. 하지만 목성과의 공명 현상으로 인해 소행성들이 목성의 중력에 영향을 받아 행성을 형성하지 못하고 분산되거나 파괴되는 경우가 많습니다. 따라서 소행성대는 행성이 형성되기 어려운 환경이라고 할 수 있습니다. 이러한 이유로 소행성대에는 행성이 없는 것입니다.


## RAG 구현

준비된 검색엔진과 LLM을 활용하셔 대화형 RAG 구현

In [22]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="",
)


In [86]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = "당신은 과학 상식 전문 AI입니다. 사용자의 메시지와 Reference 문서를 활용하여 짧게 답변합니다. Reference 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 답변합니다."

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = "당신은 과학 상식 전문 AI입니다. 과학에 관한 주제에만 tools를 호출하세요. 과학 분야가 아니라면 tools를 절대 호출하지 말고, 적절한 대답을 짧게 생성하세요."

# Function calling에 사용할 함수 정의
tools=[
        {
            "name": "search",
            "description": "Search for documents related to scientific common sense",
            "input_schema": {
                "type": "object",
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "The final query suitable for searching scientific common sense in user message history. The query is written by changing the person's name to Korean.",
                    }
                },
                "required": ["standalone_query"],
            },
        }
    ]


In [87]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    # msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=512,
            messages=messages,
            tool_choice={"type": "auto"},
            tools=tools,
            system=persona_function_calling,
            temperature=0,
            timeout=60
        )
        print(result.content)
    except Exception as e:
        print(e)
        return response

    # 검색이 필요한 경우 검색 호출후 결과를 활용하여 답변 생성
    if result.stop_reason == "tool_use":
        print("Claude wants to use a tool")

        last_content_block = result.content[-1]
        if last_content_block.type == 'tool_use':
            tool_name = last_content_block.name
            tool_inputs = last_content_block.input
            print(f"=======Claude Wants To Call The {tool_name} Tool=======")
            print(tool_inputs)
            if tool_name == "search":
                standalone_query = tool_inputs["standalone_query"]  
                print(standalone_query)
                response["standalone_query"] = standalone_query
                
                # Baseline으로는 sparse_retrieve만 사용하여 검색 결과 추출
                search_result = sparse_retrieve(standalone_query, 3)

        retrieved_context = []
        for i,rst in enumerate(search_result['hits']['hits']):
            retrieved_context.append(rst["_source"]["content"])
            response["topk"].append(rst["_source"]["docid"])
            response["references"].append({"score": rst["_score"], "content": rst["_source"]["content"]})

        content = json.dumps(retrieved_context)
        messages.append({"role": "assistant", "content": content})
        try:
            qaresult = client.messages.create(
                        model="claude-3-5-sonnet-20240620",
                        max_tokens=512,
                        system=persona_qa,
                        messages=messages,
                        temperature=0,
                        timeout=30
                    )
            print(f"QA Result: {qaresult.content}")
        except Exception as e:
            print(e)
            return response
        response["answer"] = qaresult.content[-1].text

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        print("Claude did NOT call a tool")

    return response


In [88]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            # if idx > 5:
            #   break
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            print(f'output: {output}\n')
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1


In [89]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("./data/eval.jsonl", "sample_submission.csv")

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
[TextBlock(text='나무의 분류에 대해 조사하는 것은 과학적인 주제이므로, 이에 대한 정보를 검색해보겠습니다.', type='text'), ToolUseBlock(id='toolu_01PNyEhSPozetPCeGyvUhMUo', input={'standalone_query': '나무 분류 방법 과학적 조사'}, name='search', type='tool_use')]
Claude wants to use a tool
=======Claude Wants To Call The search Tool=======
{'standalone_query': '나무 분류 방법 과학적 조사'}
나무 분류 방법 과학적 조사
QA Result: [TextBlock(text='\n\n나무의 분류를 조사하기 위한 방법은 다음과 같습니다:\n\n1. 성장 속도, 온도 범위, 크기 등 물리적 특성 관찰\n2. 잎과 꽃의 형태 비교\n3. 유전자나 단백질 구조 분석 (분자 수준 조사)\n\n이러한 방법들을 통해 나무의 종류와 특성을 파악하고 분류할 수 있습니다.', type='text')]
Answer: 

나무의 분류를 조사하기 위한 방법은 다음과 같습니다:

1. 성장 속도, 온도 범위, 크기 등 물리적 특성 관찰
2. 잎과 꽃의 형태 비교
3. 유전자나 단백질 구조 분석 (분자 수준 조사)

이러한 방법들을 통해 나무의 종류와 특성을 파악하고 분류할 수 있습니다.

output: {'eval_id': 78, 'standalone_query': '나무 분류 방법 과학적 조사', 'topk': ['c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d', '6680836e-4c14-41e8-8675-0244da839086', '9712bdf6-9419-4953-a8f1-8a4015dee986'], 'answer': '\n\n나무의 분

In [91]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("./data/eval.jsonl", "sample_submission_missing.csv")

Test 0
Question: [{'role': 'user', 'content': '우성인 빨간색 꽃과 열성인 흰꽃을 교배한다고 하자.'}, {'role': 'assistant', 'content': '네 말씀하세요.'}, {'role': 'user', 'content': '두 꽃을 교배한 후 다음세대 꽃의 색깔은 어떻게 되나?'}]
[TextBlock(text='이 질문은 유전학과 관련된 과학적 상식에 대한 것이므로, 더 정확한 정보를 찾아보겠습니다.', type='text'), ToolUseBlock(id='toolu_01N21gRJombLXQuhc4Q2FoT8', input={'standalone_query': '우성 빨간색 꽃과 열성 흰색 꽃 교배 후 다음 세대 꽃 색깔'}, name='search', type='tool_use')]
Claude wants to use a tool
=======Claude Wants To Call The search Tool=======
{'standalone_query': '우성 빨간색 꽃과 열성 흰색 꽃 교배 후 다음 세대 꽃 색깔'}
우성 빨간색 꽃과 열성 흰색 꽃 교배 후 다음 세대 꽃 색깔
QA Result: [TextBlock(text='\n\n주어진 정보에 따르면, 우성인 빨간색 꽃과 열성인 흰꽃을 교배하면 다음 세대의 꽃 색깔은 100% 빨간색이 됩니다. 이는 빨간색이 우성이기 때문에 열성인 흰색 유전자를 가리기 때문입니다.', type='text')]
Answer: 

주어진 정보에 따르면, 우성인 빨간색 꽃과 열성인 흰꽃을 교배하면 다음 세대의 꽃 색깔은 100% 빨간색이 됩니다. 이는 빨간색이 우성이기 때문에 열성인 흰색 유전자를 가리기 때문입니다.

output: {'eval_id': 54, 'standalone_query': '우성 빨간색 꽃과 열성 흰색 꽃 교배 후 다음 세대 꽃 색깔', 'topk': ['39821ad7-cabe-4ccc-a6bc-4c0a9592fa4a', 'b519a156-4

In [90]:
!wc -l sample_submission.csv

220 sample_submission.csv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
